# Entrenamiento de una GigaGAN - MGE
En este cuaderno, complementario a mi investigación sobre GigaGAN, trato de poner en práctica lo aprendido para entrenar una GigaGAN (aunque en versión reducida) con el objetivo de generar imágenes de perros, concretamente pastor alemán, a una resolución de 128x128.

Nota: Esta versión de entrenamiento será una versión "reducida" de las GigaGAN, básicamente porque no dispongo de todos los recursos de cómputo necesarios. Por ello, no haré uso de ningún CLIP Text Encoder, y simplemente será generar imagenes en base a un vector de ruido aleatorio.

## Paso 1: Instalar GigaGAN e importar

In [1]:
#!pip install gigagan-pytorch
# Descomenta el codigo de arriba si no lo tienes instalado

import os
import torch
import zipfile
import gdown
import os

from PIL import Image
from tqdm import tqdm
from gigagan_pytorch import GigaGAN, ImageDataset
from torchvision.utils import make_grid, save_image

/root/miniconda3/envs/gpu/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Paso 2: Descargar Dataset
Recuerdo que este dataset es completamente sintetico, generado localmente con FLUX.1-schnell, de black forest labs. Son 5000 imagenes 256x256 de pastores alemanes en una variedad de escenarios.

Nota: al ser tantas imagenes sinteticas no las he revisado una a una. Quizas alguna que otra imagen presenta artefactos o texto que puede perjudicar al entrenamiento de la GAN.

In [6]:
# Enlace de Kaggle al dataset
https://www.kaggle.com/datasets/warc0s/german-shepherd

# Recuerda extraerlo y guardarlo dentro de una carpeta llamada "dataset"

Descarga y extracción completadas. Archivo ZIP eliminado.


## Paso 3: Preprocesado de las imágenes

Bajaremos la resolución de cada imagen a 128x128 para acelerar el entrenamiento. No dispongo de tanto tiempo y cómputo como para poder entrenar una GigaGAN en alta resolución durante miles de épocas.

In [2]:
# Carpeta que contiene las imágenes a procesar
dataset_folder = "dataset"

# Extensiones de imagen a considerar
image_extensions = ('.jpg', '.jpeg', '.png')

# Obtener la lista de archivos de imagen en la carpeta
files = [f for f in os.listdir(dataset_folder) if f.lower().endswith(image_extensions)]

# Procesar cada imagen con un tqdm para mostrar el progreso
for filename in tqdm(files, desc="Procesando imágenes"):
    file_path = os.path.join(dataset_folder, filename)
    try:
        with Image.open(file_path) as img:
            # Redimensionar la imagen a 128x128 usando un filtro de alta calidad
            img_resized = img.resize((128, 128), Image.Resampling.LANCZOS)
            # Sobrescribir la imagen original con la versión redimensionada
            img_resized.save(file_path)
    except Exception as e:
        continue

Procesando imágenes: 100%|█████████████████████████████████████████████████████████| 5000/5000 [00:07<00:00, 688.84it/s]


## Paso 4: Comenzar el entrenamiento

Será de 100 épocas, guardando una grid de imagenes 3x3 para ver como entrena, así como un checkpoint del modelo, cada 5 épocas.

In [2]:
import os
import torch
import matplotlib.pyplot as plt
import torchvision.utils as vutils

from gigagan_pytorch import GigaGAN, ImageDataset

# Optimización de cudnn para acelerar las convoluciones (sobre todo en GPUs modernas)
torch.backends.cudnn.benchmark = True

# ================================
# Configuración del Modelo y Dataset
# ================================

# Configuración de GigaGAN adaptada a resolución 128x128.
# Se mantiene dim_max en 256 para estabilidad a esta resolución.
gan = GigaGAN(
    generator = dict(
        dim_capacity = 8,
        style_network = dict(
            dim = 64,
            depth = 4
        ),
        image_size = 128,      # Resolución 128x128
        dim_max = 256,         # Ajustado para mayor estabilidad a baja resolución
        num_skip_layers_excite = 4,
        unconditional = True
    ),
    discriminator = dict(
        dim_capacity = 16,
        dim_max = 256,         # Ajustado para mayor estabilidad
        image_size = 128,      # Resolución 128x128
        num_skip_layers_excite = 4,
        unconditional = True
    ),
    amp = False  # Desactivado, da error "NAN" para las metricas G y MSG si lo activas. Aunque acelere el entrenamiento.
).cuda()

# Configuración del dataset
dataset = ImageDataset(
    folder = 'dataset',  # Carpeta local con las imágenes
    image_size = 128     # Redimensionar imágenes a 128x128
)

# Ajustamos el batch size a 8 - Limite para mis 24GB
batch_size = 8
dataloader = dataset.get_dataloader(batch_size=batch_size)

# Asignamos el dataloader al modelo GAN.
gan.set_dataloader(dataloader)

# ================================
# Configuración del Entrenamiento
# ================================

num_epochs = 100                # Entrenamiento durante 100 épocas
epoch_steps = len(dataloader)   # Cada época recorre el dataset completo
grad_accum_every = 1            # Con batch_size de 8 no se acumulan gradientes

# Directorios para guardar las grids de imágenes y los checkpoints (cada 10 épocas)
os.makedirs('epoch_images', exist_ok=True)
os.makedirs('checkpoints', exist_ok=True)

# ================================
# Bucle de Entrenamiento
# ================================
print("=== INICIO DEL ENTRENAMIENTO ===")
for epoch in range(1, num_epochs + 1):
    print(f"\n--- Época {epoch} de {num_epochs} ---")
    
    # Se entrena durante una época completa (recorriendo el dataset)
    gan(steps=epoch_steps, grad_accum_every=grad_accum_every)
    
    # Cada 5 épocas se guarda la grid de imágenes y el checkpoint del modelo
    if epoch % 5 == 0:
        # Generamos 9 imágenes a partir de vectores de ruido aleatorio
        sample_imgs = gan.generate(batch_size=9)
        grid = vutils.make_grid(sample_imgs, nrow=3, normalize=True)
        grid_path = os.path.join("epoch_images", f"IMG_EPOCH_{epoch}.png")
        plt.imsave(grid_path, grid.permute(1, 2, 0).cpu().numpy())
        print(f"Guardada grid de la época {epoch} en: {grid_path}")
        
        # Guardamos el checkpoint del modelo
        checkpoint_path = os.path.join("checkpoints", f"checkpoint_epoch_{epoch}.pth")
        torch.save(gan.state_dict(), checkpoint_path)
        print(f"Checkpoint guardado en: {checkpoint_path}")

print("\n=== ENTRENAMIENTO FINALIZADO ===")



Generator: 12.62M
Discriminator: 15.75M


=== INICIO DEL ENTRENAMIENTO ===

--- Época 1 de 100 ---


  0%|▏                                                                                          | 1/625 [00:00<?, ?it/s]

G: 23.54 | MSG: 30.07 | VG: 0.00 | D: 5.02 | MSD: 65.11 | VD: 0.00 | GP: 0.00 | SSL: 26.74 | CL: 0.00 | MAL: 0.00


  3%|██▊                                                                               | 21/625 [00:22<09:15,  1.09it/s]

G: 277.26 | MSG: -130.05 | VG: 0.00 | D: 15.23 | MSD: 1399.48 | VD: 0.00 | GP: 16325.52 | SSL: 0.92 | CL: 0.00 | MAL: 0.00


...

62481it [06:58,  1.85it/s]

G: 1.10 | MSG: -0.76 | VG: 0.00 | D: 0.80 | MSD: 8.91 | VD: 0.00 | GP: 0.17 | SSL: 0.01 | CL: 0.00 | MAL: 0.00


62501it [07:12,  1.45it/s]

G: 0.86 | MSG: -0.64 | VG: 0.00 | D: 0.80 | MSD: 8.82 | VD: 0.00 | GP: 1.01 | SSL: 0.01 | CL: 0.00 | MAL: 0.00
complete 62500 training steps
Guardada grid de la época 100 en: epoch_images/IMG_EPOCH_100.png


Checkpoint guardado en: checkpoints/checkpoint_epoch_100.pth

=== ENTRENAMIENTO FINALIZADO ===
